# Conceptual Design Project ENG 5273
<br></br>
**Group 3**
<br></br>
The following section of the Conceptual Design Project will focus on the design of the foundations and in particular will first take a look at the initial conditions of the specific project requirements as well as the different design options and considerations we might need to explore.
<br></br>

In [1]:
from IPython.display import display, HTML
display(HTML("<style>.container { width:100% !important; }</style>"))

## Foundation types
<br></br>
**Shallow foundations**
<br></br>
Shallow foundations are a type of foundation that are used to support buildings and other structures on the surface layer of the soil. Shallow foundations are usually used for smaller and less complex Civil Engineering structures and as such are predominantly a less expensive and less complicated option, which provides easier to install and manage option suitable for small scale projects, where the soil is strong enough to support the induces loads. 
There are a multiple shallow foundation types:
* Single footings 
* Combined footings 
* Raft foundations

<br></br>
**Deep foundations**
<br></br>

Deep foundations are a type of foundation that are used to support generally speaking larger and more complex buildings and structures. Deep foundations rely on digging and excavation in order to be installed - for the purpose of reaching stronger and less deformable soil layers which can withstand the larger loads. There are several deep foundation types:
* Driven Piles
* Bored Piles
* Caissons
* Micro-Piles
* Rammed Aggregate Piers
* Grouted foundations

<br></br>

## Section 1: Introduction
<br></br>
The design of choice is Cast-in-place Group Pile foundation, also known as Displacement Continuous Flight Auger (CFA) Piles  with additional grout introduced after the construction of the reinforced concrete piles. The reasons behind this solution are the following:
* Large concentrated loads.
* To transfer the largely induced structural loads to more competent layers.
* To help resist the horizontal loads and the dynamic stresses induced by lateral wind forces.
* Limit the settlement of the structure and help integrate it in the overall structural context of the site.
* Cause minimal disturbance (noise, vibration, deformation) to the surrounding soil and structures.
* More complex shapes can be achieved.
* Protect the reinforcement from corrosion.
<br></br>

Determine whether the ULS and SLS is satisfied according to Eurocode 7 (DA1a).


## Section 2: Design Parameters

In [2]:
import math
import numpy as np
import plotly as pl
import plotly.graph_objects as go
import seaborn
import pandas
import matplotlib.pyplot as plt

In [3]:
# DESIGN FACTORS OF SAFETY

y_tan_phi = 1 # factor of safety for the angle of internal friction
y_y = 1 # material unit weight factor
y_rs = 1 # shaft capacity of the pile factor
y_rb = 1 # base capacity of the pile factor
y_rc = 1 # total capacity (compression) resistance factor

# SOIL PARAMETERS

# K = 1 and delta_angle = 0.75 phi can be assumed along the length of the pile shaft - the soil conditions are undisturbed of natural origin.

K = 1
y = 15 # [kN/m3] bulk density of the soil
phi = 32 # [degrees] angle of internal friction, then converted to radians for the calculations
phi = math.radians(phi) # [radians]

G_shear_modulus = 15e9 # [N/m2]      1GPa = 1 000 000 000 
G_si = 10.4e9 # [N/m2]      1GPa = 1 000 000 000 

v_poisson = 0.3


phi_des = np.arctan(math.tan(phi)/ y_tan_phi)
delta_angle_des = 0.75 * phi_des
cohesion_c = 0

# loading on the foundation
Q_permanent_g = 1.5e6 # [N] permanent load
Q_imposed_g = 0.5e6 # [N] imposed load
vertical_effective_stress = 1

# pile geometric properties
Lp = 10 # [m]
r_m = Lp # [m]
d_diameter_pile = 0.5 # [m]
A_pile = d_diameter_pile ** 2 # [m2], for circular use - (((d_diameter_pile / 2 )** 2) * math.pi)
C_pile = d_diameter_pile * 4 # [m], for circular use - (d_diameter_pile * math.pi)

# pile material properties
Ep = 30e9 # [N/m2]      1GPa = 1 000 000 000 

# what portion of the shaft is actively contributing
z1 = 10
z2 = 8

# pile group geometric properties
# number of piles
n_number_piles = 6
pile_group_x = 2 # [m] - dependant on the size of the pile group
pile_group_y = 6 # [m] - dependant on the size of the pile group
A_block = pile_group_x * pile_group_y
C_block = 2 * (pile_group_x + pile_group_y)

## Section 3: Formulas and Functions

In [51]:
# the following function assigns a 1.35 factor of safety for permanent loads and 1.5 for variable

def return_design_action_applied_Q_t_des(Q_permanent_g, Q_imposed_g):
    return Q_permanent_g * 1.35 + Q_imposed_g + 1.5

def return_total_ultimate_resistance_Qt(Qs, Qb):
    return Qs + Qb

def return_ultimate_base_resistance_Qb(D, qbu):
    return math.pi * (D ** 2) * qbu / 4


# the alpha method - short term undrained load-carrying capacity, where the alpha factor is responding 
# to the smoothness of the pile surface - soil interface, alpha = 0, means perfectly smooth, while
# alpha = 1, means perfectly rough surface
# su - the undrained shear strength, not fundamental property, but related to the angle of internal 
# friction phi

def return_t_ult(alpha, su):
    return alpha * su

# the beta method - drained long-term conditons, following coloumb's law.
# this design consideration is applicable if only the water table is below the tip of the piles, which is 
# the case on the JWS-Rankine site

def return_pile_group_ultimate_capacity(y, phi, cohesion_c=0):
    pass
    
def return_design_base_capacity_Q_bu_des(A_pile, N_q, Lp, y, y_y, y_rb): # [N]
    return A_pile * N_q * Lp * (y / y_y) / y_rb * 1000

def return_design_shaft_capacity_Q_su_des(C_pile, K, y, y_y, z1, z2, delta_angle_des,  y_rs): # [N]
    return (C_pile * K * math.tan(delta_angle_des) * y / y_y * ((z1 * z1 / 2) - (z2 * z2 /2)) / y_rs) * 1000


# BLOCK GROUP FUNCTIONS
def return_design_base_capacity_BLOCK_Q_b_block_des(A_block, N_q, Lp, y, y_y, y_rb): # [N]
    return A_block * N_q * Lp * (y / y_y) / y_rb * 1000

def return_design_shaft_capacity_BLOCK_Q_s_block_des(C_block, K, y, y_y, z1, z2, delta_angle_des,  y_rs): # [N]
    return (C_block * K * math.tan(delta_angle_des) * y / y_y * ((z1 * z1 / 2) - (z2 * z2 /2)) / y_rs) * 1000



# All of these come from Janbu(1976)
def return_bearing_capacity_factor_N_q(phi_des):
    return (((1 + math.sin(phi_des)) / (1 - math.sin(phi_des))) * math.exp(math.pi * math.tan(phi_des)))

def return_cohesion_factor_N_c(N_q, phi_des):
    return (N_q - 1) / math.tan(phi_des)

def return_gamma_factor_N_y(N_q, phi_des_des):
    return (N_q - 1) * math.tan(phi)

def return_q_bu():
    pass



# ULS DESIGN CHECK

def check_if_individual_pile_design_capacity_satisfies_ULS(n_number_piles):
    if ((Q_bu_des + Q_su_des) * n_number_piles) >= Q_t_des:
        return True
    else:
        return False
    
    
def check_if_BLOCK_design_capacity_satisfies_ULS(n_number_piles):
    if ((Q_b_block_des + Q_s_block_des)) >= Q_t_des:
        return True
    else:
        return False
    
    
# SLS DESIGN CHECK Randolph & Wroth (1978) closed form solution

# SLS DESIGN SINGLE PILE

def return_stiffness_of_the_base_Kbi(D, G_shear_modulus, v):
    return 2 * D * G_shear_modulus / (1 - v)

def return_shear_stress(shear_stress_average, D, r):
    return  D * shear_stress_average / (2 * r)

def return_stiffness_of_the_shaft_Ksi(Lp, G_si, r_m, D):
    return 2 * math.pi * Lp * G_si / (math.log(2 * r_m / D))

def return_single_pile_settlement(Q, Ksi, Kbi):
    return Q / (Kbi + Ksi)

def return_load_carried_by_the_shaft_Ksi_s(s, Ksi):
    return s * Ksi

def return_load_carried_by_the_shaft_Kbi_s(s, Kbi):
    return s * Kbi


# SLS DESIGN PILE GROUP 

def return_mu(G_si, r_m, D, Ep, Ap):
    return (math.sqrt(2 * math.pi * G_si / (math.log(2 * r_m / D) * Ep * Ap)))

def return_omega(Kbi, Ep, A_pile, mu):
    return Kbi / (Ep * A_pile * mu)



ULS DESIGN CHECK
<br></br>

In [33]:
N_q = return_bearing_capacity_factor_N_q(phi)
N_c = return_cohesion_factor_N_c(N_q, phi)

print(N_q)
print(N_c)


23.17677620701264
35.49026070689835


In [34]:

print(N_q)

Q_t_des = return_design_action_applied_Q_t_des(Q_permanent_g, Q_imposed_g)
Q_bu_des = return_design_base_capacity_Q_bu_des(A_pile, N_q, Lp, y, y_y, y_rb)
print(Q_bu_des)

Q_su_des = return_design_shaft_capacity_Q_su_des(C_pile, K, y, y_y, z1, z2, delta_angle_des,  y_rs)

print(Q_su_des)

23.17677620701264
1868627.581690394
240423.49006660955


Design of group pile foundation procedure for n-number of piles - the respective ULS, SLS.
<br></br>
First we will take a look at the ULS - especially the axial loading and the bearing capacity the combined number of piles as well as of the pile group as whole.
<br></br>
Failure Mode 1:

In [35]:



    
print(check_if_individual_pile_design_capacity_satisfies_ULS(n_number_piles))



True


Failure Mode 2:

In [36]:

Q_b_block_des = return_design_base_capacity_BLOCK_Q_b_block_des(A_block, N_q, Lp, y, y_y, y_rb)

Q_s_block_des = return_design_shaft_capacity_BLOCK_Q_s_block_des(C_block, K, y, y_y, z1, z2, delta_angle_des,  y_rs)

print(Q_s_block_des)
print(Q_b_block_des)


1923387.9205328764
89694123.92113891


In [37]:

    
print(check_if_BLOCK_design_capacity_satisfies_ULS(n_number_piles))


True


SLS DESIGN CHECK FOR A SINGLE PILE
<br></br>


In [60]:
d_diameter_pile = 0.5
G_shear_modulus = 15.75e6
v_poisson = 0.2
Lp = 21.50
G_si = 10.4e6
r_m = Lp
Q = 40 # [MN]
Ep = 30e9
A_pile = math.pi * (d_diameter_pile ** 2) / 4


Kbi = (return_stiffness_of_the_base_Kbi(d_diameter_pile, G_shear_modulus, v_poisson) / 1000000)
Ksi = (return_stiffness_of_the_shaft_Ksi(Lp, G_si, r_m, d_diameter_pile) / 1000000)
mu = return_mu(G_si, r_m, d_diameter_pile, Ep, A_pile)
omega = return_omega(Kbi, Ep, A_pile, mu) * 1000000

print(f"The stiffness of a Single Pile at the base:  Kbi = {Kbi:.3f} [MN/m].")
print(f"The stiffness of a Single Pile at the shaft:  Ksi = {Ksi:.3f} [MN/m].")

print(f"The settlement of a Single Pile:  s = {(return_single_pile_settlement(Q, Ksi, Kbi) * 1000):.3f} [mm].")
print(f"Mu = {mu:.3f}")
print(f"Omega = {omega:.3f}")

The stiffness of a Single Pile at the base:  Kbi = 19.688 [MN/m].
The stiffness of a Single Pile at the shaft:  Ksi = 315.404 [MN/m].
The settlement of a Single Pile:  s = 119.370 [mm].
Mu = 0.050
Omega = 0.067


SLS DESIGN CHECK FOR GROUP PILE
<br></br>

## Section 4: Results and Design Sizing

## Section 5: Additional Considerations